### feature합치기
#### investors, historical(매일 거래량 등) and 분류항목 합치기

In [1]:
import datetime, time
from datetime import date

import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler

In [2]:
# index column : date
def find_previous_date(df_inv, df_his, date_current): # 현재일에서 바로 직전 거래일을 찾음.
    i = 0
    date2 = 0
    while True:
        i += 1
        date2 = date_current - datetime.timedelta(days = i)

        if (df_inv.loc[date2]['retail'] != 0) & (df_his.loc[date2]['vol'] != '') :  
            # investors와 historical 두 개 데이터에 같이 없어야 함
            break
        if i >= 30:  # 현재일 대비 최대 30일전까지 거래가 있을 경우 계산 (현실적으로 최대 10일 이상 폐장되는 경우 없음)
            break
            
    return date2

In [3]:
# 거래일과 거래일 바로 이전일의 변화율을 계산함.
def find_ratio(df_o, date_current):
    df_o_trans = df_o.transpose()
    df_o_trans.columns = ['before', 'after']
    df_o_trans[date_current] = (df_o_trans['after'] - df_o_trans['before']) / df_o_trans['before']
    return df_o_trans.transpose()

In [4]:
# 변화율(historical, investors), weekday를 계산하고 합하여 return함
def combine_data(df_o, df_inv, df_his):
    # df_inv : df_investor, df_his: df_historical
    investor_rate = pd.DataFrame()
    historical_rate = pd.DataFrame()
    date_weekday = pd.DataFrame()
    
    # 최초의 0이 아닌(최초 거래가 이루어진 일자) 날짜로 부터 일주일 후부터 기준.
    # 최초 2일전까지의 날짜를 확보하기 위함. 현재일의 전날과 전전날의 이틀간 데이터 변화율 계산
    for k, i in enumerate(df_inv.iloc[:, 1]):  
        if i != 0: 
            skip_num = k + 8 
            break

    for date_current in df_o.index[skip_num:]: # 최초 2일전까지의 날짜를 확보하기 위함. 현재일의 전날과 전전날의 이틀간 데이터 변화율 계산
        date_previous = find_previous_date(df_inv, df_his, date_current)
        # 거래 전날 요일 구하기
        date_temp = {'date': date_current, 'weekday' : date_previous.weekday()}
        df_temp = pd.DataFrame(date_temp, index=[0]).set_index('date')
        date_weekday = pd.concat([date_weekday, df_temp], axis=0)
        date_previous_1 = find_previous_date(df_inv, df_his, date_previous)
        # 거래 전날, 전전날을 확인하고 변화정도 계산하기(find_ratio)
        df_inv_comp = df_inv.loc[[date_previous_1, date_previous]]
        df_his_comp = df_his.loc[[date_previous_1, date_previous]]
        investor_rate = pd.concat([investor_rate, find_ratio(df_inv_comp, date_current).iloc[[-1]]], axis=0)
        historical_rate = pd.concat([historical_rate, find_ratio(df_his_comp, date_current).iloc[[-1]]], axis=0)
    total = pd.concat([investor_rate, historical_rate, date_weekday, df_o ], axis=1)
    return total

In [5]:
code = {'005930': ['삼성전자', 'sec'], '005380': ['현대차', 'hyunmotor'],
                 '035420': ['NAVER', 'naver'], '033780': ['KT&G', 'ktng']}
# code = {'005930': ['삼성전자', 'sec']}

code = {'005930' : ['삼성전자', 'sec'], '373220' : ['LG에너지솔루션', 'lgenergy'], 
        '000660' : ['SK하이닉스', 'skhinix'], '207940' : ['삼성바이오로직스', 'ssbio'],
        '006400' : ['삼성SDI', 'sdi'], '051910' : ['LG화학', 'lgchemical'],
        '005935' : ['삼성전자우', 'secpre'], '005380' : ['현대차', 'hyunmotor'],
        '035420' : ['NAVER', 'naver'], '000270' : ['기아','kia'],
        '035720' : ['카카오', 'kakao'], '005490' : ['POSCO홀딩스', 'poscoholding'],
        '105560' : ['KB금융', 'kbbank'], '028260' : ['삼성물산', 'sscnt'],
        '068270' : ['셀트리온', 'celltrion'], '012330' : ['현대모비스', 'mobis'],
        '055550' : ['신한지주', 'shgroup'], '066570' : ['LG전자', 'lgelec'],
        '003670' : ['포스코케미칼', 'poscochemical'], '096770' : ['SK이노베이션', 'skinnovation'],
        '033780' : ['KT&G', 'ktng']}
# code = {'035420' : ['NAVER', 'naver']}

In [6]:
# hist_column = [ 'date', 'open', 'high', 'low', 'close', 'close_cr', 'vol']
hist_column_m = [ 'date', 'open', 'high', 'low', 'close', 'vol'] # close_cr 제외하고 사용. divided by zero 회피용
inv_column = ['date', 'retail', 'foreigner', 'institution', 'financial', 'invtrust', 'pension', 'privequity',
              'bank', 'insurance', 'financeetc', 'corporateetc', 'foreigneretc']

In [7]:
# investor.pkl, historical.pkl. 읽기

# ****************************************************************************************************
## 주요: 오늘날짜로 데이터를 분석하고자 하면 pkl directory 내의 .pkl 데이터가 오늘 날짜로 모두 update시켜 놓아야 함.
##   C:\jupyter연습\factors_affecting_stock_price' 디렉토리에 있는
#    'get_company_daily_price_since_last_update_to_pickle.ipynb' 과 (historical.pkl과 investors.pkl update 프로그램)
#     'get_common_data_since_last_update_to_pickle (미국, 한국 주요 지표 등 update용)
# ****************************************************************************************************
directory_for_ml = '../data/data_for_ml/'
pkl_directory = '../data/company_pkl/'

# 전체 column을 선정하여, 분석시에 선별하여 사용하도록 함.
column_selected = ['retail', 'foreigner', 'institution', 'financial', 'invtrust', 'pension', 'privequity',
              'bank', 'insurance', 'financeetc', 'corporateetc', 'foreigneretc',
              'open', 'high', 'low', 'close', 'vol','weekday']

column_selected.extend(['cr_00', 'cr_05', 'cr_10', 'cr_15', 'cr_20'])  # 아래 class column 이 변경에 따라 수정해야 함

for key, val in code.items():
#     pkl_name= '{}{}_4class.pkl'.format(directory_for_ml, val[1])
#     df_class_temp = pd.read_pickle(pkl_name)
#     df_class_temp['date'] = pd.to_datetime(df_class_temp['date']) #change to timestamp
#     df_class_temp['date'] = df_class_temp['date'].dt.date  # change to datetime
 
    pkl_name= '{}_historical.pkl'.format(val[1])
    df_historical_temp = pd.read_pickle(pkl_directory + pkl_name)
    df_historical_temp = df_historical_temp[hist_column_m]
    # close_cr 행을 없앰(변동이 없는 경우가 빌생하여 divided zero error 발생), close_cr은 target column에서 재 계산하여사용
    df_historical_temp['date'] = df_historical_temp['date'].dt.date # change to datetime
    
    pkl_name= '{}_investors.pkl'.format(val[1])
    df_investors_temp = pd.read_pickle(pkl_directory + pkl_name)
    df_investors_temp['date'] = df_investors_temp['date'].dt.date # change to datetime
    
    # ******** 시작 일자, 마지막 일자  지정 ***********
    start_date = datetime.date(2022, 1, 1) # 2022년 01월 01일 자료 있음. 추후 이전날짜 추가시 수정 필요
    end_date = df_investors_temp['date'].iloc[-1]  # 투자자별 자료가 있는 마지막 날짜
    
    date_range_ts = pd.date_range(start=start_date, end=end_date)
    df_base = pd.DataFrame(pd.Series(date_range_ts, name='date'))
    df_base['date'] = df_base['date'].dt.date
    
# get combined data (eg. df_sec_combined, df_ktng_combined...)
#     df_combined_temp = combine_data(df_class_temp.set_index('date'), df_investors_temp.set_index('date'),
#                                     df_historical_temp.set_index('date'))

    df_combined_temp = combine_data(df_base.set_index('date'), 
                                    df_investors_temp.set_index('date'), df_historical_temp.set_index('date'))
    
    min_rate = 0.0 # +로 끝난 상황을 알기 위함
    df_combined_temp['cr_00'] = df_combined_temp['close'].map(lambda x : 0 if x <= min_rate else ( 1 if x > min_rate  else  "" ))
    min_rate = 0.005 # 수수료등 비용 0.2672% 이상 확인하기 위함, 0.5% 상승 마감
    df_combined_temp['cr_05'] = df_combined_temp['close'].map(lambda x : 0 if x <= min_rate else ( 1 if x > min_rate  else  "" ))
    min_rate = 0.010 # 1.0% 상승 마감
    df_combined_temp['cr_10'] = df_combined_temp['close'].map(lambda x : 0 if x <= min_rate else ( 1 if x > min_rate  else  "" ))
    min_rate = 0.015 # 1.5% 상승 마감
    df_combined_temp['cr_15'] = df_combined_temp['close'].map(lambda x : 0 if x <= min_rate else ( 1 if x > min_rate  else  "" ))
    min_rate = 0.020 # 2.0% 상승 마감
    df_combined_temp['cr_20'] = df_combined_temp['close'].map(lambda x : 0 if x <= min_rate else ( 1 if x > min_rate  else  "" ))
    
    globals()['df_{}_{}'.format(val[1], 'combined')] = df_combined_temp.copy()
    globals()['df_{}_{}'.format(val[1], 'sel')] = df_combined_temp[column_selected]

In [8]:
'''
# 전체 column을 선정하여, 분석시에 선별하여 사용하도록 함.
column_selected = ['retail', 'foreigner', 'institution', 'financial', 'invtrust', 'pension', 'privequity',
              'bank', 'insurance', 'financeetc', 'corporateetc', 'foreigneretc',
              'open', 'high', 'low', 'close', 'vol','weekday']
'''

"\n# 전체 column을 선정하여, 분석시에 선별하여 사용하도록 함.\ncolumn_selected = ['retail', 'foreigner', 'institution', 'financial', 'invtrust', 'pension', 'privequity',\n              'bank', 'insurance', 'financeetc', 'corporateetc', 'foreigneretc',\n              'open', 'high', 'low', 'close', 'vol','weekday']\n"

## 주요 지표 읽기

### Read and Combine data 

In [9]:
# common_pkl = ["bok_rate.pkl", "cpi.pkl", "dji.pkl", "dji_future.pkl", "dxy_future.pkl", 
#               "fear_greed.pkl", "fed_rate.pkl", "gold.pkl", "ixic_future.pkl", "kor_10yr_bond.pkl",
#               "kor_2yr_bond.pkl", "kosdaq.pkl", "kospi.pkl", "krw_rate.pkl", "nas.pkl",
#               "snp_future.pkl", "sox.pkl", "spx.pkl", "us_10yr_bond.pkl", "us_2yr_bond.pkl",
#               "us_3mon_bond.pkl", "vix.pkl", "wti_future.pkl"]

In [10]:
# 매일 변화가 있는 항목 추출; 금리변동과 같이 주기적을 발생하는 항목들(cpi, bok_rate, fed_rate 등)은 제외
common_pkl = [ "dji.pkl", "dji_future.pkl", "dxy_future.pkl", 
               "ixic_future.pkl", "kor_10yr_bond.pkl",
              "kor_2yr_bond.pkl", "kosdaq.pkl", "kospi.pkl", "krw_rate.pkl", "nas.pkl",
              "snp_future.pkl", "sox.pkl", "spx.pkl", "us_10yr_bond.pkl", "us_2yr_bond.pkl",
              "us_3mon_bond.pkl", "vix.pkl", "wti_future.pkl"]

In [11]:
end_date = datetime.datetime.today().date()
start_date = '20220101'  # 2022년 01월 01일 자료 있음. 추후 이전날짜 추가시 수정 필요
date_range_ts = pd.date_range(start=start_date, end=end_date)

In [12]:
pkl_common_directory = '../data/common_pkl/'
df_base = pd.DataFrame(pd.Series(date_range_ts, name='date')).set_index('date')
# for index_name in major_index:
for index_name in common_pkl:
    key_name = index_name[:-4]
#     globals()['df_{}'.format(key_name)] = pd.read_pickle(pkl_common_directory + index_name)
#     df_base = df_base.merge(globals()['df_{}'.format(key_name)].set_index('date').iloc[:, [-1]], \
#                             how='left', left_index=True, right_index=True)
    df_temp = pd.read_pickle(pkl_common_directory + index_name)
    df_temp['date'] = df_temp['date'] + datetime.timedelta(days = 1) # 하루전 데이터와 일치시키기 위하여 하루 shift
    df_base = df_base.merge(df_temp.set_index('date').iloc[:, [-1]], \
                            how='left', left_index=True, right_index=True)

In [13]:
'''
df_sec_sel = df_sec_combined[column_selected]
'''

'\ndf_sec_sel = df_sec_combined[column_selected]\n'

In [14]:
for key, val in code.items():
    globals()['df_{}_sel'.format(val[1])] \
    = globals()['df_{}_sel'.format(val[1])].merge(df_base, how='left', left_index=True, right_index=True)

In [15]:
'''
df_sec_sel = df_sec_sel.merge(df_base, how='left', left_index=True, right_index=True)
df_hyunmotor_sel = df_hyunmotor_sel.merge(df_base, how='left', left_index=True, right_index=True)
df_naver_sel = df_naver_sel.merge(df_base, how='left', left_index=True, right_index=True)
df_ktng_sel = df_ktng_sel.merge(df_base, how='left', left_index=True, right_index=True)
'''

"\ndf_sec_sel = df_sec_sel.merge(df_base, how='left', left_index=True, right_index=True)\ndf_hyunmotor_sel = df_hyunmotor_sel.merge(df_base, how='left', left_index=True, right_index=True)\ndf_naver_sel = df_naver_sel.merge(df_base, how='left', left_index=True, right_index=True)\ndf_ktng_sel = df_ktng_sel.merge(df_base, how='left', left_index=True, right_index=True)\n"

In [16]:

# 전체 column을 선정하여, 분석시에 선별하여 사용하도록 함.
# weekday, 'cr_00', 'cr_05', 'cr_10', 'cr_15', 'cr_20' column을 마지막으로 이동 
new_columns = ['retail', 'foreigner', 'institution', 'financial', 'invtrust',
                'pension', 'privequity', 'bank', 'insurance', 'financeetc',
                'corporateetc', 'foreigneretc', 
                'dji_cr', 'dji_f_cr', 'ixic_cr', 'ixic_f_cr', 'spx_cr', 'spx_f_cr',
                'bond_kor_10_cr', 'bond_kor_2_cr', 'dxy_cr', 'bond_usa_10_cr','bond_usa_2_cr', 'bond_usa_3m_cr',
                'kosdaq_cr', 'kospi_cr', 'krw_cr', 'sox_cr', 'vix_cr', 'wti_cr', 
#                 'open', 'high', 'low', 'close', 'vol', 'weekday',  # close column 제거함 (class column 계산항목으로 사용됨)
                'open', 'high', 'low', 'vol', 'weekday', 
                'cr_00', 'cr_05', 'cr_10', 'cr_15', 'cr_20']

In [17]:
# column 순서 변경 : weekday, 'cr_00', 'cr_05', 'cr_10', 'cr_15', 'cr_20' column을 마지막으로 이동 
for key, val in code.items():
    globals()['df_{}_sel'.format(val[1])] = globals()['df_{}_sel'.format(val[1])][new_columns]

In [18]:
'''
# drop inf, -inf : replace inf to 1 or -1 . 데이터를 살리고
# 변동률이 무한대가 되는 것을 방지하기 위해서, 나중에 발생하는 에러를 방지
df_sec_sel.replace([np.inf, -np.inf], [1, -1], inplace=True)

.impute 사용하는 것을 고려할 필요 있음.
'''

'\n# drop inf, -inf : replace inf to 1 or -1 . 데이터를 살리고\n# 변동률이 무한대가 되는 것을 방지하기 위해서, 나중에 발생하는 에러를 방지\ndf_sec_sel.replace([np.inf, -np.inf], [1, -1], inplace=True)\n\n.impute 사용하는 것을 고려할 필요 있음.\n'

In [19]:
# drop inf, -inf : replace inf to 1 or -1 . 데이터를 살리고
# 변동률이 무한대가 되는 것을 방지하기 위해서, 나중에 발생하는 에러를 방지
for key, val in code.items():
    globals()['df_{}_sel'.format(val[1])].replace([np.inf, -np.inf], [1, -1], inplace=True)

In [20]:
# # delete rows which include NaN : dji, spx, nasdaq 지수가 nan인 rows 제거
# df_sec_sel.dropna(inplace=True)

In [21]:
# delete rows which include NaN : dji, spx, nasdaq 지수가 nan인 rows 제거
for key, val in code.items():
    globals()['df_{}_sel'.format(val[1])].dropna(inplace=True)

In [22]:
# string column을 numeric으로 전환
def string_to_num(df):
    df.replace('%', '', regex=True, inplace=True)
    return df.apply(pd.to_numeric)
'''
df_sec_sel = string_to_num(df_sec_sel)
'''

for key, val in code.items():
    globals()['df_{}_sel'.format(val[1])] = string_to_num(globals()['df_{}_sel'.format(val[1])])

In [23]:
# pickle로 데이터 저장
'''
directory_for_ml = '../data/data_for_ml/'
df_sec_sel.to_pickle(directory_for_ml + 'df_sec_sel.pkl')
df_sec_sel.to_csv(directory_for_ml + 'df_sec_sel.csv')
'''

for key, val in code.items():
    globals()['df_{}_sel'.format(val[1])].to_pickle(directory_for_ml + 'df_{}_sel.pkl'.format(val[1]))
    globals()['df_{}_sel'.format(val[1])].to_csv(directory_for_ml + 'df_{}_sel.csv'.format(val[1]))

### feature 이름 설명
#### feature에 사용된 column name 설명

In [ ]:
df_feature_name_abbr = pd.read_csv('../data/feature_name_abbr.csv')

In [ ]:
standforK = lambda x : df_feature_name_abbr['Korean'][df_feature_name_abbr['abbr'] == x].values[0]
standforE = lambda x : df_feature_name_abbr['English'][df_feature_name_abbr['abbr'] == x].values[0]
standforexp = lambda x : df_feature_name_abbr['exp1'][df_feature_name_abbr['abbr'] == x].values[0]

In [ ]:
# financial이 무엇인지 알기 위한 명령
c_name = 'dxy_cr'
standforK(c_name), standforE(c_name), standforexp(c_name)

In [ ]:
c_name = 'cpi'
standforK(c_name), standforE(c_name), standforexp(c_name)

In [ ]:
c_name = 'sox'
standforK(c_name), standforE(c_name), standforexp(c_name)

### 설명에서 feature name 찾기

In [ ]:
find_col_nameK = lambda x : df_feature_name_abbr[['abbr', 'Korean']][df_feature_name_abbr['Korean'].str.contains(x)]
find_col_nameE = lambda x : df_feature_name_abbr[['abbr', 'English']][df_feature_name_abbr['English'].str.contains(x, na=False, case=False)]

In [ ]:
e_word = 'INDEX' # English column에서 찾기
find_col_nameE(e_word)

In [ ]:
k_word = '반도체' # Korean column에서 찾기
find_col_nameK(k_word)

#### pd.merge 형식 참조
df_base = df_base.merge(djis, how='left', left_index=True, right_index=True)

In [ ]:
bokd = '../data/common_pkl/bok_rate.pkl'
cpid = '../data/common_pkl/cpi.pkl'
djid = '../data/common_pkl/dji.pkl'
bok = pd.read_pickle(bokd)
cpi = pd.read_pickle(cpid)
dji = pd.read_pickle(djid)
boks = pd.read_pickle(bokd).set_index('date')
cpis = pd.read_pickle(cpid).set_index('date')
djis = pd.read_pickle(djid).set_index('date')

In [ ]:
end_date = datetime.datetime.today().date()
start_date = '20220620'
date_range_ts = pd.date_range(start=start_date, end=end_date)

In [ ]:
df_base = pd.DataFrame(pd.Series(date_range_ts, name='date')).set_index('date')

In [ ]:
df_base = df_base.merge(cpis, how='left', left_index=True, right_index=True)

In [ ]:
df_base = df_base.merge(djis, how='left', left_index=True, right_index=True)